In [17]:
!rm -r mesa-keras-rl
!git clone https://github.com/eescriba/mesa-keras-rl
!cd mesa-keras-rl/ && pip install -e .


rm: cannot remove 'mesa-keras-rl': No such file or directory
Cloning into 'mesa-keras-rl'...
remote: Enumerating objects: 429, done.
remote: Counting objects: 100% (429/429), done.
remote: Compressing objects: 100% (312/312), done.
remote: Total 429 (delta 118), reused 389 (delta 82), pack-reused 0
Receiving objects: 100% (429/429), 811.82 KiB | 12.68 MiB/s, done.
Resolving deltas: 100% (118/118), done.
ERROR: File "setup.py" not found. Directory cannot be installed in editable mode: /content


In [21]:
import gym
import numpy as np

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Flatten, Embedding, Reshape
from tensorflow.keras.optimizers import Adam

from rl.agents.dqn import DQNAgent
from rl.policy import EpsGreedyQPolicy
from rl.memory import SequentialMemory


from mobility.taxinet.env import TaxiNetEnv

In [24]:
env = TaxiNetEnv(file_path="mesa-keras-rl/src/envs/mobility/resources/city.txt")
env.seed(123)
action_size = env.action_space.n
env.reset()

31850

In [25]:
model = Sequential()
model.add(Embedding(45000, 10, input_length=1))
model.add(Reshape((10,)))
model.add(Dense(4500, activation='relu'))
model.add(Dense(4500, activation='relu'))
model.add(Dense(4500, activation='relu'))
model.add(Dense(action_size, activation='linear'))
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 1, 10)             450000    
_________________________________________________________________
reshape_1 (Reshape)          (None, 10)                0         
_________________________________________________________________
dense_4 (Dense)              (None, 4500)              49500     
_________________________________________________________________
dense_5 (Dense)              (None, 4500)              20254500  
_________________________________________________________________
dense_6 (Dense)              (None, 4500)              20254500  
_________________________________________________________________
dense_7 (Dense)              (None, 6)                 27006     
Total params: 41,035,506
Trainable params: 41,035,506
Non-trainable params: 0
__________________________________________

In [ ]:
memory = SequentialMemory(limit=50000, window_length=1)
policy = EpsGreedyQPolicy()
dqn = DQNAgent(model=model, nb_actions=action_size, memory=memory, nb_steps_warmup=500, target_model_update=1e-2, policy=policy)
dqn.compile(Adam(lr=1e-3), metrics=['mae'])

In [ ]:
dqn.fit(env, nb_steps=1000000, visualize=False, verbose=1, nb_max_episode_steps=99, log_interval=100000)

In [ ]:
dqn.save_weights('dqn_{}_weights.h5f'.format(ENV_NAME), overwrite=True)

In [ ]:
dqn.test(env, nb_episodes=5 , visualize=False, nb_max_episode_steps=99)